In [1]:
import pandas as pd

In [2]:
train = pd.read_csv('train.csv', sep = '\t', index_col='id') #memory error
test = pd.read_csv('test.csv', sep='\t', encoding='utf8')
unsupervised = pd.read_csv('other.csv', sep='\t', encoding='utf8')

In [8]:
from bs4 import BeautifulSoup
import re
#from nltk.corpus import stopwords

def text_to_wordlist(text):   
    
    text = re.sub('[^А-ЯЁа-яё]', ' ', text)
    words = text.lower().split()

    #stops = set(stopwords.words("english"))

    return words

In [14]:
split_train = train['description'].apply(lambda x: text_to_wordlist(x) )

In [15]:
split_train.head()

id
0    [в, новый, магазин, хозтовары, приглашаем, на,...
1    [обязанности, работа, на, токарных, станках, с...
2    [требования, опыт, программирования, на, с, зн...
3    [условия, на, работу, вахтовым, методом, в, г,...
4    [компания, палладиум, стандарт, признанный, ли...
Name: description, dtype: object

In [6]:
import nltk.data

def text_to_sentences(text):
    text = BeautifulSoup(text).get_text()
    
    tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
    raw_sentences = tokenizer.tokenize(text.strip())
    
    sentences = []
    for raw_sentence in raw_sentences:
        if len(raw_sentence) > 0:
            sentences.append(text_to_wordlist(raw_sentence))

    return sentences

In [16]:
sentences = []
for description in train["description"]:
    sentences += text_to_sentences(description)

/home/katya/.pyenv/versions/katya_bank_project/lib/python3.5/site-packages/bs4/__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 193 of the file /home/katya/.pyenv/versions/3.5.4/lib/python3.5/runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup(YOUR_MARKUP})

to this:

 BeautifulSoup(YOUR_MARKUP, "lxml")

  markup_type=markup_type))


In [20]:
%%time
for description in unsupervised["description"]:
    sentences += text_to_sentences(description)

/home/katya/.pyenv/versions/katya_bank_project/lib/python3.5/site-packages/bs4/__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 193 of the file /home/katya/.pyenv/versions/3.5.4/lib/python3.5/runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup(YOUR_MARKUP})

to this:

 BeautifulSoup(YOUR_MARKUP, "lxml")

  markup_type=markup_type))


CPU times: user 14min 4s, sys: 18.2 s, total: 14min 22s
Wall time: 14min 22s


In [21]:
%%time
for description in test["description"]:
    sentences += text_to_sentences(description)

/home/katya/.pyenv/versions/katya_bank_project/lib/python3.5/site-packages/bs4/__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 193 of the file /home/katya/.pyenv/versions/3.5.4/lib/python3.5/runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup(YOUR_MARKUP})

to this:

 BeautifulSoup(YOUR_MARKUP, "lxml")

  markup_type=markup_type))


CPU times: user 4min 24s, sys: 5.46 s, total: 4min 29s
Wall time: 4min 29s


In [22]:
print('\t Первый элемент массива')
print(sentences[0])
print()
print('\t Второй элемент массива')
print(sentences[1])

	 Первый элемент массива
['в', 'новый', 'магазин', 'хозтовары', 'приглашаем', 'на', 'постоянную', 'работу', 'руководителя', 'секции', 'хозтовары']

	 Второй элемент массива
['обязанности', 'организация', 'эффективного', 'и', 'культурного', 'обслуживания', 'покупателей', 'организация', 'приемки', 'сдачи', 'товаров', 'на', 'склад', 'консультация', 'покупателей', 'по', 'вопросам', 'касающимся', 'оказываемых', 'услуг', 'контроль', 'своевременной', 'подачи', 'товаров', 'в', 'торговую', 'секцию', 'проверка', 'качества', 'сроков', 'годности', 'тмц', 'наличие', 'маркировок', 'ценников', 'на', 'товарах', 'организация', 'бесперебойной', 'работы', 'товарной', 'секции', 'и', 'участка', 'склада', 'закрепленного', 'за', 'ней', 'контроль', 'за', 'сохранностью', 'товаров', 'торгового', 'оборудования', 'и', 'прочих', 'материальных', 'ценностей', 'инвентаризация', 'товаров', 'требования', 'знание', 'товарной', 'группы', 'знание', 'правил', 'приемки', 'товара', 'опыт', 'работы', 'в', 'розничной', 'торгов

In [27]:
%%time
# симортируем соответствующую функцию из модуля gensim, который должен быть установлен

from gensim.models.word2vec import Word2Vec 

# список параметров, которые можно менять по вашему желанию
num_features = 300  # итоговая размерность вектора каждого слова
min_word_count = 5  # минимальная частотность слова, чтобы оно попало в модель
num_workers = 20     # количество ядер вашего процессора, чтоб запустить обучение в несколько потоков
context = 10        # размер окна 
downsampling = 1e-3 # внутренняя метрика модели

model = Word2Vec(sentences, workers=num_workers, size=num_features,
                 min_count=min_word_count, window=context, sample=downsampling)

CPU times: user 1h 3min 25s, sys: 10.9 s, total: 1h 3min 36s
Wall time: 12min 12s


In [29]:
# Финализируем нашу модель. Ее нельзя будет доучить теперь, но она начнет занимать гораздо меньше места# Финал 
model.init_sims(replace=True)

In [33]:
model.most_similar('хозтовары')

/home/katya/.pyenv/versions/katya_bank_project/lib/python3.5/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('канцтовары', 0.7524976134300232),
 ('канцелярия', 0.6687374114990234),
 ('посуда', 0.650962233543396),
 ('бумага', 0.62480628490448),
 ('одноразовая', 0.6224671602249146),
 ('канцелярские', 0.6138722896575928),
 ('полотенца', 0.6121525168418884),
 ('утварь', 0.6102863550186157),
 ('вода', 0.6019569635391235),
 ('хоз', 0.5966950058937073)]

In [34]:
model.wv.index2word

['и',
 'в',
 'с',
 'на',
 'работы',
 'по',
 'опыт',
 'обязанности',
 'условия',
 'требования',
 'работа',
 'от',
 'для',
 'компании',
 'знание',
 'график',
 'оформление',
 'к',
 'мы',
 'рф',
 'до',
 'тк',
 'образование',
 'не',
 'контроль',
 'за',
 'возможность',
 'работать',
 'продаж',
 'плата',
 'заработная',
 'умение',
 'обучение',
 'высшее',
 'клиентов',
 'или',
 'лет',
 'ведение',
 'ответственность',
 'роста',
 'работу',
 'день',
 'наличие',
 'товара',
 'карьерного',
 'соответствии',
 'навыки',
 'трудоустройство',
 'организация',
 'официальное',
 'участие',
 'оплата',
 'месяц',
 'профессионального',
 'проведение',
 'пк',
 'г',
 'документации',
 'подготовка',
 'вы',
 'из',
 'сотрудников',
 'года',
 'компания',
 'при',
 'вас',
 'пакет',
 'рабочий',
 'работ',
 'разработка',
 'оборудования',
 'полный',
 'оклад',
 'желание',
 'документов',
 'выполнение',
 'у',
 'продукции',
 'о',
 'работе',
 'сфере',
 'а',
 'пользователь',
 'покупателей',
 'офис',
 'территории',
 'клиентами',
 'согласн

In [39]:
import numpy as np

def text_to_vec(words, model, size):
    text_vec = np.zeros((size,), dtype="float32")
    n_words = 0

    index2word_set = set(model.wv.index2word)
    for word in words:
        if word in index2word_set:
            n_words = n_words + 1
            text_vec = np.add(text_vec, model[word])
    
    if n_words != 0:
        text_vec /= n_words
    return text_vec

In [36]:
def texts_to_vecs(texts, model, size):
    texts_vecs = np.zeros((len(texts), size), dtype="float32")
    
    for i, text in enumerate(texts):
        texts_vecs[i] = text_to_vec(text, model, size)

    return texts_vecs

In [37]:
# действительно работает, сделаем для всех текстов из train
train_like_word_list = [sum(text_to_sentences(text), []) for text in train['description']]

/home/katya/.pyenv/versions/katya_bank_project/lib/python3.5/site-packages/bs4/__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 193 of the file /home/katya/.pyenv/versions/3.5.4/lib/python3.5/runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup(YOUR_MARKUP})

to this:

 BeautifulSoup(YOUR_MARKUP, "lxml")

  markup_type=markup_type))


In [40]:
train_vecs = texts_to_vecs(train_like_word_list, model, num_features)

/home/katya/.pyenv/versions/katya_bank_project/lib/python3.5/site-packages/ipykernel_launcher.py:11: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  # This is added back by InteractiveShellApp.init_path()


In [41]:
# сделаем тоже самое для test
test_like_word_list = [sum(text_to_sentences(text), []) for text in test['description']]

/home/katya/.pyenv/versions/katya_bank_project/lib/python3.5/site-packages/bs4/__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 193 of the file /home/katya/.pyenv/versions/3.5.4/lib/python3.5/runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup(YOUR_MARKUP})

to this:

 BeautifulSoup(YOUR_MARKUP, "lxml")

  markup_type=markup_type))


In [42]:
test_vecs = texts_to_vecs(test_like_word_list, model, num_features)

/home/katya/.pyenv/versions/katya_bank_project/lib/python3.5/site-packages/ipykernel_launcher.py:11: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  # This is added back by InteractiveShellApp.init_path()


In [45]:
from sklearn.ensemble import RandomForestClassifier

forest = RandomForestClassifier(n_estimators=100, n_jobs=8)
forest = forest.fit(train_vecs, train["target"])
predict = forest.predict_proba(test_vecs)

In [46]:
predict

array([[ 0.11,  0.89],
       [ 0.  ,  1.  ],
       [ 0.02,  0.98],
       ..., 
       [ 0.56,  0.44],
       [ 0.99,  0.01],
       [ 1.  ,  0.  ]])

In [47]:
submission = pd.DataFrame({"id":test["id"],
                           "target":predict[:,1]})

In [48]:
submission.head()

,id,target
0,200000,0.89
1,200001,1.00
2,200002,0.98
3,200003,1.00
4,200004,0.04


In [49]:
submission.to_csv("hw_06_submit_w2v.csv", 
                  index=False)